In [20]:
heaviside(t) = 0.5 * (sign.(t) .+ 1)
J_box = w -> real(2*V^2*sqrt.(Complex.(1 .-w.^2))/pi)
J_ell = w -> ((V^2)/2)*(heaviside(w .+ 1) .- heaviside(w .- 1))  



function direct_mapping(f,Nb,ϵi,Ns)
    samp = 100 # Number of points in mesh.
    x = LinRange(-1,1,samp);
    y = LinRange(-1,1,Nb+1)
    tsq =  Vector{Float64}(undef,Nb)
    en =  Vector{Float64}(undef,Nb)
    for i =1:Nb
        x = LinRange(y[i],y[i+1],samp)
        Jx =f(x)
        tsq[i] = trapz(x,Jx); 
        en[i] = (1/tsq[i])*trapz(x,x.*Jx);
    end
    ind = sortperm(abs.(en.-ϵi[Ns]))                               # Broadcast abs() over vector using "." notation.
        tsq, en = tsq[ind], en[ind];  
    return [tsq,en] 
end

  
function reaction_mapping(f,Nb)
   
   # J = J*(heaviside(w+1) - heaviside(w-1)); # Ensure input function is clipped to the domain.
    
    #Define fixed numerical mesh over [-2,2] to capture spectral function and
    # its hilbert transform correctly within [-1,1].
    samp = 1000 # Number of points in mesh.
    x = LinRange(-2,2,samp);
    
    Jx =f(x)  # Evaluate symbolic input function over the grid.

    tsq = zeros(1,Nb)
    en = zeros(1,Nb)
    # Loop over the omega intervals and perform integrations:
    Jcur = Jx; # Current bath spectral function.
    for s=1:Nb
      
      # Simple trapezoid integration for hopping squared and on-site energy:
      tsq[s] = trapz(x,Jcur); 
      en[s] = (1/tsq[s])*trapz(x,x.*Jcur);

      Jprev = Jcur;
      JH = imag(hilbert(Jprev)); # Hilbert transform.
      Jcur = ((tsq[s]/pi)^2)*Jprev./(JH.^2+Jprev.^2);
    end
    
    return [tsq, en]
end
    
    

function band_diag(B,d)
# Band-diagonalize matrix B with a bandwidth of d:
    n = size(B,1); # Assumed to be square.
    U = Diagonal(ones(n,n));
    for k=1:Int(floor(n/d)-1)
        C = B[(k*d+1):n,((k-1)*d+1):(k*d)]; # Extract coupling matrix.
        F = qr(C); # Upper-triangularize.
        blocks = [[Diagonal(ones(k*d,k*d))]; [F.Q']]
        Q = cat(blocks...,dims=(1,2))    # Form full triangularizing unitary.    
        B = Q*B*Q'; # Apply to input matrix to transform for next step.
        U = Q*U; # Save this step's unitary to the full sequence.
    end
    return B,U; # Return the final band-diagonalized matrix.
end;

function U_thermo(N,f_k)
    U_th = zeros(N,N)
    U_th[1,1],U_th[2,2] = 1,1
    b=0
    for i=3:2:N
        b += 1
        U_th[i,i],U_th[i+1,i+1] = sqrt(1-f_k[b]),-sqrt(1-f_k[b])
        U_th[i,i+1],U_th[i+1,i] = sqrt(f_k[b]),sqrt(f_k[b])
    end
    return U_th
end

function U_chain(N,U1,U2)

    U_tot = zeros(N,N)
    U_tot[1,1],U_tot[2,2] = 1,1
    b1 = 0
    for i=3:2:N
        b2 = 0 # resets the column iteration
        b1 +=1
        for j =3:2:N
            b2 += 1

            U_tot[i,j] = U1[b1,b2]
            U_tot[i+1,j+1] = U2[b1,b2]
        end
    end
    return U_tot
end

function initialise_psi(choice,f_k,Ns)
    """
    ERROR: The tags of psi are incorrect due to this function.
    """
    
    
    
    if choice == 1
        gates = [(sqrt(f_k[Int((n-1)/2)])*cd[n]*Id[n+1] + sqrt(1-f_k[Int((n-1)/2)])*Id[n]*cd[n+1])/sqrt(2) for n in 3:2:N]; 
    end

    if choice==2 || choice==3
         gates = [cd[n]*Id[n+1] for n in (2*Ns+1):2:N]

    end

    system_gate = [(cd[n]*Id[n+1] + Id[n]*cd[n+1])/sqrt(2) for n in 1:2:(2*Ns-1)]

    if N>2*Ns
        for i in reverse(1:length(system_gate))
            push!(gates,system_gate[i])
        end
    else
        gates = system_gate
    end
    ψ = apply(gates,vac;cutoff=1e-15);
    return ψ
end


function H_(N,Ns,ϵi,V_k,ϵb,choice,s)
    #The following code is not optimised, various objects are created multiple times
    #and within each different option there is identical code which doesn't
    #need to be written multiple times.
    
    ###Create Hamiltonian MPO 
    terms = OpSum()
    ###Create single particle matrix hamiltonian
    H_single = zeros(N,N)

    ### For all choices, the first two indices are the system ancilla and the system respectively.
    for i=1:Ns

        terms += ϵi[i],"n",2*i;
        H_single[2*i,2*i] = ϵi[i]
    end
    b = 0


    if choice ==1
        print("negative ancilla H = -1, no ancilla H = 0, positive ancilla H = 1")
        HA_choice = parse(Int,readline()) 
        model =  "E basis,"
        if HA_choice ==1
            model = model*"Ha=Hb"
        end
        if HA_choice ==0
            model = model*"Ha=0"
        end
        if HA_choice ==-1
            model = model*"Ha=-1"
        end
        for j=(2*Ns+1):2:N
            b += 1
            terms += ϵb[b],"n",j                                   # bath mode self energy
            H_single[j,j] = ϵb[b]

            terms += HA_choice*ϵb[b],"n",j+1                       # ancilla bath mode self energy
            H_single[j+1,j+1] =  HA_choice*ϵb[b]

            terms += V_k[b],"Cdag",j,"C",2*Ns                         #hopping from system to kth f mode 
            H_single[j,2*Ns] = V_k[b]

            terms += conj(V_k[b]),"Cdag",2*Ns,"C",j                   #hopping from kth f mode to system 
            H_single[2*Ns,j] = conj(V_k[b])
        end    
    end

    if choice==2
        model = "thermofield basis"
        for j=(2*Ns+1):2:N
            b += 1
            terms += ϵb[b],"n",j                                   # filled mode self energy
            H_single[j,j] = ϵb[b]

            terms += ϵb[b],"n",j+1                                 # empty mode self energy
            H_single[j+1,j+1] =  ϵb[b]

            terms += V_k[b]*sqrt(f_k[b]),"Cdag",j,"C",2*Ns            #hopping from system to kth f mode
            H_single[j,2*Ns] = V_k[b]*sqrt(f_k[b])

            terms += conj(V_k[b])*sqrt(f_k[b]),"Cdag",2*Ns,"C",j      #hopping from kth f mode to system 
            H_single[2*Ns,j] = conj(V_k[b])*sqrt(f_k[b])

            terms += V_k[b]*sqrt(1-f_k[b]),"Cdag",j+1,"C",2*Ns        #coupling from system to kth e mode
            H_single[j+1,2*Ns] =  V_k[b]*sqrt(1-f_k[b])

            terms += conj(V_k[b])*sqrt(1-f_k[b]),"Cdag",2*Ns,"C",j+1  #coupling from kth e mode to system
            H_single[2*Ns,j+1] =  conj(V_k[b])*sqrt(1-f_k[b])
        end
    end

    if choice ==3
        model = "thermofield+tridiag"
        fill_mat = zeros(Nb+1,Nb+1)
        emp_mat = zeros(Nb+1,Nb+1)
        ### Same terms as for choice 2, inputed as a matrix rather than an MPO. 
        for j=1:Nb
            ###This loop creates two (Nb+1) x (Nb+1) matrices, one includes the couplings and self energies of the 
            ###filled modes and the system, the other the empty modes and the system. These are then tridiagonalised 
            ###separately in the next loop, and their elements are used to construct the MPO for the hamiltonian in this new, tridiagonal basis. 

            fill_mat[1,1],emp_mat[1,1] = ϵi[Ns], ϵi[Ns]

            fill_mat[j+1,j+1],emp_mat[j+1,j+1] = ϵb[j],ϵb[j]

            fill_mat[j+1,1] = V_k[j]*sqrt(f_k[j])

            fill_mat[1,j+1] = conj(V_k[j])*sqrt(f_k[j])

            emp_mat[j+1,1] = V_k[j]*sqrt(1-f_k[j])

            emp_mat[1,j+1] = conj(V_k[j])*sqrt(1-f_k[j])
        end
        heatmap(fill_mat)
        heatmap(emp_mat)
        fill_mat,Uf = band_diag(fill_mat,1)
        emp_mat,Ue = band_diag(emp_mat,1)

        Uf = Uf[2:Nb+1,2:Nb+1] ###discarding the system terms as the system isn't mixed in this transformation.
        Ue = Ue[2:Nb+1,2:Nb+1]

        b = 1
        for j=(2*Ns+1):2:N
            b += 1
            terms += fill_mat[b,b],"n",j
            H_single[j,j] = fill_mat[b,b]

            terms += emp_mat[b,b],"n",j+1
            H_single[j+1,j+1] = emp_mat[b,b]

            terms += emp_mat[b-1,b],"Cdag",j-1,"C",j+1
            H_single[j-1,j+1] = emp_mat[b-1,b]

            terms += emp_mat[b,b-1],"Cdag",j+1,"C",j-1
            H_single[j+1,j-1] = emp_mat[b,b-1]
            if j==2*Ns+1
                terms += fill_mat[b-1,b],"Cdag",2*Ns,"C",2*Ns+1
                H_single[2*Ns,2*Ns+1] = fill_mat[b-1,b]

                terms += fill_mat[b,b-1],"Cdag",2*Ns+1,"C",2*Ns
                H_single[2*Ns+1,2*Ns] = fill_mat[b,b-1]
            else
                terms += fill_mat[b-1,b],"Cdag",j-2,"C",j
                H_single[j-2,j] = fill_mat[b-1,b]

                terms += fill_mat[b,b-1],"Cdag",j,"C",j-2
                H_single[j,j-2] = fill_mat[b,b-1]
            end
        end
    end

    return  MPO(terms,s),H_single
end


H_ (generic function with 1 method)

In [5]:
LinRange(-1,1,3)

3-element LinRange{Float64, Int64}:
 -1.0,0.0,1.0